# Noise Sensitivity and Error Analysis

## Objective

Evaluate model robustness to uncertainty through:
- Price noise sensitivity
- Yield noise sensitivity
- Mean Square Error (MSE) analysis

## Methodology

Introduce noise: $P_{it} = \bar{P}_{it}(1 + \epsilon_p)$, $Y_{it} = \bar{Y}_{it}(1 + \epsilon_y)$

where $\epsilon_p, \epsilon_y \sim N(0, \sigma^2)$

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB

np.random.seed(2024)
plt.rcParams['font.size'] = 10

## 1. Load Baseline Predictions

In [2]:
# Load Problem 3 predictions
pred_df = pd.read_csv('crop_predictions_problem3.csv')

CROPS = pred_df['Crop'].unique()
YEARS = sorted(pred_df['Year'].unique())
N_CROPS = len(CROPS)

# Build parameter dictionaries
price_base = {}
cost_base = {}
yield_base = {}
demand_base = {}

for _, row in pred_df.iterrows():
    i, t = row['Crop'], row['Year']
    price_base[i, t] = row['Price_mean']
    cost_base[i, t] = row['Cost_mean']
    yield_base[i, t] = row['Yield_per_Mu_mean']
    demand_base[i, t] = row['Sales_Volume_mean']

print(f"Loaded {N_CROPS} crops, {len(YEARS)} years")

Loaded 41 crops, 7 years


## 2. Simplified Optimization Function

In [3]:
def optimize_with_noise(price, cost, yield_pm, demand, time_limit=120):
    """
    Run optimization with given (possibly noisy) parameters.
    Returns total profit.
    """
    # Load plot data
    farmland_df = pd.read_excel('Attachment1_EN.xlsx', sheet_name='Existing Village Farmland')
    farmland_df = farmland_df.dropna(subset=['Plot Name'])
    farmland_df = farmland_df[farmland_df['Plot Name'].str.match(r'^[A-F]\d+$', na=False)]
    
    PLOTS = sorted(farmland_df['Plot Name'].tolist())
    SEASONS = ['Single Season', 'First Season', 'Second Season']
    
    area = dict(zip(farmland_df['Plot Name'], farmland_df['Plot Area (Mu)']))
    plot_type = dict(zip(farmland_df['Plot Name'], farmland_df['Plot Type']))
    
    # Build compatibility (simplified)
    crops_df = pd.read_excel('Attachment1_EN.xlsx', sheet_name='Village Crops')
    crop_type_map = dict(zip(crops_df['Crop Name'], crops_df['Crop Type']))
    
    grains = [c for c in CROPS if crop_type_map.get(c) in ['Grain', 'Grain (Legumes)']]
    veg = [c for c in CROPS if crop_type_map.get(c) in ['Vegetable', 'Vegetable (Legumes)']]
    mushrooms = [c for c in CROPS if crop_type_map.get(c) == 'Edible Mushroom']
    
    COMPAT = {}
    for j in PLOTS:
        ptype = plot_type[j]
        if ptype in ['Flat Dry Land', 'Terraced Field', 'Hillside Land']:
            for i in grains:
                if i != 'Rice':
                    COMPAT[i, j, 'Single Season'] = 1
        elif ptype in ['Irrigated Land', 'Regular Greenhouse', 'Smart Greenhouse']:
            for i in veg:
                COMPAT[i, j, 'First Season'] = 1
            for i in (mushrooms if ptype == 'Regular Greenhouse' else veg):
                COMPAT[i, j, 'Second Season'] = 1
    
    VALID = [(i, j, s) for (i, j, s), v in COMPAT.items() if v == 1]
    
    # Model
    model = gp.Model("Noise_Test")
    model.setParam('OutputFlag', 0)
    model.setParam('TimeLimit', time_limit)
    model.setParam('MIPGap', 0.02)
    
    x = model.addVars([(i, j, s, t) for (i, j, s) in VALID for t in YEARS], vtype=GRB.BINARY)
    y = model.addVars(CROPS, YEARS, vtype=GRB.CONTINUOUS)
    
    # Objective
    revenue = gp.quicksum(price[i, t] * y[i, t] for i in CROPS for t in YEARS)
    costs = gp.quicksum(cost[i, t] * area[j] * x[i, j, s, t] for (i, j, s, t) in x.keys())
    model.setObjective(revenue - costs, GRB.MAXIMIZE)
    
    # Constraints (simplified)
    for j in PLOTS:
        for t in YEARS:
            model.addConstr(gp.quicksum(x[i, j, s, t] for i in CROPS for s in SEASONS if (i, j, s, t) in x) <= 1)
    
    for i in CROPS:
        for t in YEARS:
            prod = gp.quicksum(yield_pm[i, t] * area[j] * x[i, j, s, t] 
                              for j in PLOTS for s in SEASONS if (i, j, s, t) in x)
            model.addConstr(y[i, t] <= prod)
            model.addConstr(y[i, t] <= demand[i, t])
    
    model.optimize()
    
    return model.objVal if model.Status in [GRB.OPTIMAL, GRB.TIME_LIMIT] else None

# Test baseline
baseline_profit = optimize_with_noise(price_base, cost_base, yield_base, demand_base)
print(f"Baseline profit: ¥{baseline_profit:,.0f}")

Set parameter Username
Set parameter LicenseID to value 2735705
Academic license - for non-commercial use only - expires 2026-11-10
Baseline profit: ¥89,665,952


## 3. Price Noise Analysis

In [9]:
noise_levels = [0.00, 0.02, 0.05, 0.08, 0.10, 0.15, 0.20]
n_replications = 20

price_noise_results = []

for sigma in noise_levels:
    profits = []
    
    for rep in range(n_replications):
        # Add noise to prices
        price_noisy = {}
        for (i, t), p in price_base.items():
            noise = np.random.normal(0, sigma)
            price_noisy[i, t] = p * (1 + noise)
        
        profit = optimize_with_noise(price_noisy, cost_base, yield_base, demand_base, time_limit=90)
        if profit:
            profits.append(profit)
    
    mean_profit = np.mean(profits)
    std_profit = np.std(profits)
    mse = np.mean([(p - baseline_profit)**2 for p in profits])
    
    price_noise_results.append({
        'sigma': sigma,
        'mean_profit': mean_profit,
        'std_profit': std_profit,
        'mse': mse,
        'change_pct': (mean_profit - baseline_profit) / baseline_profit * 100
    })
    
    print(f"  σ={sigma:.2f}: Mean=¥{mean_profit:,.0f}, MSE={mse:,.0f}")

df_price_noise = pd.DataFrame(price_noise_results)

  σ=0.00: Mean=¥89,665,952, MSE=0
  σ=0.02: Mean=¥89,726,362, MSE=87,538,522,871
  σ=0.05: Mean=¥89,853,564, MSE=416,116,381,395
  σ=0.08: Mean=¥90,133,989, MSE=1,262,137,106,597
  σ=0.10: Mean=¥91,058,974, MSE=2,875,492,829,207
  σ=0.15: Mean=¥92,519,367, MSE=11,869,101,968,844
  σ=0.20: Mean=¥92,885,735, MSE=13,281,434,482,949


## 4. Yield Noise Analysis

In [5]:
yield_noise_results = []

for sigma in noise_levels:
    profits = []
    
    for rep in range(n_replications):
        # Add noise to yields
        yield_noisy = {}
        for (i, t), y in yield_base.items():
            noise = np.random.normal(0, sigma)
            yield_noisy[i, t] = max(y * (1 + noise), 0)  # Non-negative
        
        profit = optimize_with_noise(price_base, cost_base, yield_noisy, demand_base, time_limit=90)
        if profit:
            profits.append(profit)
    
    mean_profit = np.mean(profits)
    std_profit = np.std(profits)
    mse = np.mean([(p - baseline_profit)**2 for p in profits])
    
    yield_noise_results.append({
        'sigma': sigma,
        'mean_profit': mean_profit,
        'std_profit': std_profit,
        'mse': mse,
        'change_pct': (mean_profit - baseline_profit) / baseline_profit * 100
    })
    
    print(f"  σ={sigma:.2f}: Mean=¥{mean_profit:,.0f}, MSE={mse:,.0f}")

df_yield_noise = pd.DataFrame(yield_noise_results)

Testing yield noise sensitivity...
  σ=0.00: Mean=¥89,665,952, MSE=0
  σ=0.02: Mean=¥89,570,927, MSE=126,174,411,148
  σ=0.05: Mean=¥89,877,989, MSE=301,753,029,184
  σ=0.08: Mean=¥89,726,929, MSE=423,948,827,088
  σ=0.10: Mean=¥90,187,806, MSE=1,111,783,607,693
  σ=0.15: Mean=¥90,451,759, MSE=1,850,911,762,951
  σ=0.20: Mean=¥90,966,851, MSE=4,601,866,443,493


## 5. Sensitivity Coefficient Calculation

In [6]:
# Calculate sensitivity as dProfit/dSigma (numerical derivative)
df_price_noise['sensitivity'] = 0.0
df_yield_noise['sensitivity'] = 0.0

for i in range(1, len(df_price_noise)):
    d_profit = df_price_noise.loc[i, 'mean_profit'] - df_price_noise.loc[i-1, 'mean_profit']
    d_sigma = df_price_noise.loc[i, 'sigma'] - df_price_noise.loc[i-1, 'sigma']
    df_price_noise.loc[i, 'sensitivity'] = d_profit / d_sigma if d_sigma > 0 else 0

for i in range(1, len(df_yield_noise)):
    d_profit = df_yield_noise.loc[i, 'mean_profit'] - df_yield_noise.loc[i-1, 'mean_profit']
    d_sigma = df_yield_noise.loc[i, 'sigma'] - df_yield_noise.loc[i-1, 'sigma']
    df_yield_noise.loc[i, 'sensitivity'] = d_profit / d_sigma if d_sigma > 0 else 0

print("\nPrice Noise Results")
print(df_price_noise[['sigma', 'mean_profit', 'mse', 'sensitivity']].to_string(index=False))

print("\nYield Noise Results")
print(df_yield_noise[['sigma', 'mean_profit', 'mse', 'sensitivity']].to_string(index=False))


Price Noise Results
 sigma  mean_profit          mse  sensitivity
  0.00 8.966595e+07 0.000000e+00 0.000000e+00
  0.02 8.969600e+07 1.434403e+11 1.502329e+06
  0.05 8.998962e+07 4.505940e+11 9.787318e+06
  0.08 9.033711e+07 9.812569e+11 1.158310e+07
  0.10 9.061227e+07 2.041798e+12 1.375817e+07
  0.15 9.211407e+07 8.288073e+12 3.003585e+07
  0.20 9.337928e+07 1.768361e+13 2.530432e+07

Yield Noise Results
 sigma  mean_profit          mse   sensitivity
  0.00 8.966595e+07 0.000000e+00  0.000000e+00
  0.02 8.957093e+07 1.261744e+11 -4.751273e+06
  0.05 8.987799e+07 3.017530e+11  1.023541e+07
  0.08 8.972693e+07 4.239488e+11 -5.035352e+06
  0.10 9.018781e+07 1.111784e+12  2.304390e+07
  0.15 9.045176e+07 1.850912e+12  5.279041e+06
  0.20 9.096685e+07 4.601866e+12  1.030185e+07
